En aquesta pràctica treballareu amb un model de detecció d’objectes basat en PyTorch i la xarxa neuronal VGG16. L'objectiu es modificar el model per adaptar-lo a un problema de detecció d'objectes específic utilitzant un conjunt de dades simples.

![Exemple](08_Detecció/imgs/img.png)

Emprarem un *dataset* de detecció d'objectes que conté imatges d'estrelles. Aquest conjunt de dades és senzill i ideal per a practicar tècniques de detecció d'objectes. El podeu trobar a Kaggle al següent [enllaç](https://www.kaggle.com/datasets/kishanj/simple-object-detection). Per carregar aquest tipus de dataset haurem d'implementar una classe personalitzada que hereti de `torch.utils.data.Dataset`.

In [29]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

class EstrellesDataset(Dataset):
    def __init__(self, csv_file, img_dir, transforms=None):
        """
        Args:
            csv_file (str): Path al fitxer CSV amb anotacions.
            img_dir (str): Carpeta on hi ha les imatges.
            transforms (callable, optional): Transformacions per aplicar a les imatges.
        """
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transforms

    def __len__(self):
        # Retorna el nombre d'imatges al dataset
        return len(self.annotations)

    def __getitem__(self, idx):
        # Obté la fila corresponent
        row = self.annotations.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")

        # Extreu la bounding box i l'etiqueta
        bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
        label = row['label']

        # Si tens transformacions, les apliques
        if self.transform:
            image = self.transform(image)

        return image, label, bbox


## Preparació del model

Començarem carregant el model VGG16 preentrenat i adaptant-lo per a la detecció d'objectes. Afegirem capes addicionals per predir les caixes delimitadores (bounding boxes) i les classes dels objectes.

In [30]:
import torch
from torch import nn
from torchvision import models

# modelo VGG16 preentrenado (ya lo tienes en tu código)
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
backbone = vgg16.features  # las capas convolucionales

class VGG16ObjectDetector(nn.Module):
    def __init__(self, backbone, num_classes=2):
        """
        backbone: las features convolucionales (vgg16.features)
        num_classes: número de clases (incluyendo background si aplica)
        """
        super().__init__()
        self.backbone = backbone

        # Reutilizamos el avgpool de la VGG preentrenada
        self.avgpool = vgg16.avgpool

        # Reutilizamos las capas fully-connected de VGG excepto la última (la que produce 1000)
        # Éstas transforman el flatten de 25088 a un vector de 4096
        # list(vgg16.classifier.children()) es:
        # [Linear(25088,4096), ReLU, Dropout, Linear(4096,4096), ReLU, Dropout, Linear(4096,1000)]
        # Quitamos la última Linear para quedarnos con salida 4096
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(*list(vgg16.classifier.children())[:-1])  # output: 4096-dim

        # Cabeza de clasificación: de 4096 -> num_classes (logits)
        self.class_head = nn.Linear(4096, num_classes)

        # Cabeza de regresión de bounding box: de 4096 -> 4 (xmin, ymin, xmax, ymax)
        self.bbox_head = nn.Linear(4096, 4)

        # Opcional: inicialización pequeña para las cabezas nuevas
        nn.init.normal_(self.class_head.weight, mean=0.0, std=0.01)
        nn.init.constant_(self.class_head.bias, 0.0)
        nn.init.normal_(self.bbox_head.weight, mean=0.0, std=0.001)
        nn.init.constant_(self.bbox_head.bias, 0.0)

    def forward(self, x):
        """
        x: tensor [B, 3, H, W]
        Retorna:
            class_logits: [B, num_classes]
            bbox_preds:   [B, 4]  (en el mismo orden que usemos: xmin, ymin, xmax, ymax)
        """
        # 1) backbone convolucional
        x = self.backbone(x)          # e.g. [B, 512, H', W']
        x = self.avgpool(x)           # [B, 512, 7, 7] -> depende del tamaño de entrada
        x = self.flatten(x)           # [B, 512*7*7 = 25088]

        # 2) fully-connected (reutilizadas de VGG excepto la última capa)
        feat = self.fc(x)             # [B, 4096]

        # 3) cabezas separadas
        class_logits = self.class_head(feat)  # [B, num_classes]
        bbox_preds = self.bbox_head(feat)     # [B, 4]

        return class_logits, bbox_preds

# Ejemplo de instanciación
model = VGG16ObjectDetector(backbone=backbone, num_classes=2)


In [31]:
from PIL import Image
from torchvision import transforms
import torch

# Transformaciones como en el entrenamiento
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Cargar una imagen de prueba
img_path = "/content/sample_data/a (10).jpg"   # 🔹 cámbiala por la ruta real
image = Image.open(img_path).convert("RGB")
x = transform(image).unsqueeze(0)  # [1, 3, 224, 224]


In [32]:
model.eval()
with torch.no_grad():
    class_logits, bbox_preds = model(x)


In [33]:
# Obtener probabilidades de clase
probs = torch.softmax(class_logits, dim=1)
pred_class = torch.argmax(probs, dim=1).item()
confidence = probs[0][pred_class].item()

# Obtener bounding box
bbox = bbox_preds[0].tolist()

print(f"Pred class: {pred_class}, prob: {confidence:.3f}")
print(f"Pred bbox: {bbox}")


Pred class: 0, prob: 0.682
Pred bbox: [-0.13005539774894714, 0.04336685687303543, 0.017337141558527946, 0.020896198228001595]
